In [44]:
import pandas as pd
import re
import string
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [18]:
def clean_html(text):
    # Fungsi Menghapuss HTML & url
    # Menghapus entitas html (seperti &amp;) jika ada
    text = re.sub(r'\&\w*;', '', text)

    # Menghapus URL
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b', '', text)

    return text

def remove_puncts(text):
    # Fungsi Menghapus tanda baca
    puncts = string.punctuation

    # Menghapus tanda baca
    text = re.sub('[' + puncts + ']+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)  # removing repeating punctuations

    return text

In [37]:
def clean_text(text):
    # Fungsi manipulasi teks / memberssihkan teks


    # Menghapus spasi di awal kalimat
    text = text.lstrip(' ')

    # Menghapus whitespace
    text = re.sub(r'\s\s+', ' ', text)

    # menghapus RT @username 
    text = re.sub(r'@\S*', '', text)
    text = re.sub(r'RT', '', text)

    # menghapus unicode
    text = re.sub(r'[^\x00-\x7F]+', '', text)


    text = text.lower()
    # Menghapus kata yang terdiri dari <2 huruf
    # text = re.sub(r'\b\w{1,2}\b', '', text)

    return text

In [38]:
data = pd.read_csv('dataset_penyisihan_bdc_2024.csv', sep=';')

In [39]:
# clean
data['cleaned_text'] = data['text'].apply(clean_text)
data['cleaned_text'] = data['cleaned_text'].apply(clean_html)
data['cleaned_text'] = data['cleaned_text'].apply(remove_puncts)

In [72]:
with open('preprocess/stopwords.txt', 'r') as f:
    stopwords = f.read().splitlines()

with open('preprocess/stopwords.txt', 'r') as f:
    slang_text = f.read()
  
slang_list = slang_text.splitlines()

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Cleaning stopwords dari teks
def clean_stopwords(text):
  words = text.split()
  texts = [word for word in words if word not in stopwords]
  return texts

# Mengubah slangwords ke kata baku
def clean_slangwords(text):
  texts = [slang_list[slang_list.index(word)+1] if word in slang_list else word for word in text]
  return texts

# Stemming kata ke bentuk kata dasar
def stem(text):
  stemmed_text = [stemmer.stem(word) for word in text]
  return stemmed_text

In [73]:
data['tokenized_text'] = data['cleaned_text']
data['tokenized_text'] = data['tokenized_text'].apply(clean_stopwords)
data['tokenized_text'] = clean_slangwords(data['tokenized_text'])
data['tokenized_text'] = data['tokenized_text'].apply(stem)

In [74]:
# save to csv
!del cleaned_dataset_penyisihan_bdc_2024.csv
data.to_csv('cleaned_dataset_penyisihan_bdc_2024.csv', sep=';', index=False)